In [1]:
from src.active_learner import active_learner

In [218]:
al_random = active_learner(n_initial_samples = 2448,
                    n_epochs = [1,1,1,1,1],
                    low_certainty_threshold = 1000,
                    low_certainty_method = 'Random',
                    softmax_uncertainty_method ='entropy',
                    al_until_full_train_set = False,
                    BS = 256,
                    RS = 42
                   )

In [219]:
al_softmax = active_learner(n_initial_samples = 2448,
                    n_epochs = [1,1,1,1,1],
                    low_certainty_threshold = 1000,
                    low_certainty_method = 'Softmax',
                    softmax_uncertainty_method ='entropy',
                    al_until_full_train_set = False,
                    BS = 256,
                    RS = 42
                   )

In [220]:
np.random.seed(RS)
torch.manual_seed(RS)
al_random._do_training(al_random.n_epochs[0])

starting training process.

training epoch [1]
Percent: [###################################] 100% train_loss: 5.324 train_acc: 12.5 Done...
loss_train_avg: 5.467, train_acc_avg: 5.039
starting validation process
Percent: [###################################] 100% Done...
loss_val_avg: 5.268, val_acc_avg: 8.387
Start testing model
Percent: [###################################] 100% Done...
Model scored 8.4% on Validation Set


In [221]:
np.random.seed(RS)
torch.manual_seed(RS)
al_softmax._do_training(al_softmax.n_epochs[0])

starting training process.

training epoch [1]
Percent: [###################################] 100% train_loss: 5.31 train_acc: 11.81 Done...
loss_train_avg: 5.465, train_acc_avg: 4.852
starting validation process
Percent: [###################################] 100% Done...
loss_val_avg: 5.265, val_acc_avg: 8.159
Start testing model
Percent: [###################################] 100% Done...
Model scored 8.17% on Validation Set


In [222]:
all(al_random.train_loader.sampler.indices == al_softmax.train_loader.sampler.indices)

True

In [223]:
set(al_random.val_loader.sampler.indices) == set(al_softmax.val_loader.sampler.indices)

True

In [224]:
al_softmax_dict = {'train_loader': al_softmax.train_loader,
                   'val_loader': al_softmax.val_loader,
                   'test_loader': al_softmax.test_loader}
al_random_dict = {'train_loader': al_random.train_loader,
                   'val_loader': al_random.val_loader,
                   'test_loader': al_random.test_loader}

In [225]:
print("Element-wise:")
for loader in ['train_loader', 'val_loader', 'test_loader']:
    print(loader,
al_softmax_dict[loader].sampler.indices == al_random_dict[loader].sampler.indices)

Element-wise:
train_loader [ True  True  True ...  True  True  True]
val_loader [ True  True  True ...  True  True  True]
test_loader [ True  True  True ...  True  True  True]


In [226]:
print("Set-wise:")
for loader in ['train_loader', 'val_loader', 'test_loader']:
    print(loader,
set(al_softmax_dict[loader].sampler.indices) == set(al_random_dict[loader].sampler.indices))

Set-wise:
train_loader True
val_loader True
test_loader True


In [234]:
np.random.seed(RS)
torch.manual_seed(RS)
al_softmax._add_samples()

Starting to predict for Softmax uncertainty measurement using entropy
Percent: [###################################] 100% Done...

1000 least confident samples have been added to the trainingsset
0 most confident samples have been added to the trainingsset

Doing Training Cycle with 4448 labeled and 0 pseudo-labeled samples (18% of total dataset labeled)


In [235]:
np.random.seed(RS)
torch.manual_seed(RS)
al_random._add_samples()


1000 least confident samples have been added to the trainingsset
0 most confident samples have been added to the trainingsset

Doing Training Cycle with 4448 labeled and 0 pseudo-labeled samples (18% of total dataset labeled)


In [236]:
al_softmax.log['current_train_indices'] == al_random.log['current_train_indices']

array([False, False, False, ..., False, False,  True])

In [238]:
len(set(al_softmax.log['current_train_indices'])-set(al_random.log['current_train_indices']))/len(al_softmax.log['current_train_indices'])

0.4084982014388489

In [231]:
def _do_training(self, n_epochs, optim_args = {'lr': 0.0001}):

    #set the trainloader.samples to the current selected ones
    self.train_loader.sampler.indices = self.log['current_train_indices']

    #set the validation laoder to the right samples. This is crucial since we misused the val_loader instance for the AL_Validation
    self.val_loader.sampler.indices = self.log['current_val_indices']

    #Create a new model and solver
    self.model = custom_resnet18(num_classes=257, p=0.5, MCD = self.MCD)
    self.solver = solver(optim_args = optim_args)

    #use the solver to train the model
    self.solver.train(self.model, self.train_loader, self.val_loader, num_epochs= n_epochs, show_intermediate_steps = 10)

    #Safe the Epoch with best Validation accuracy and acc
    self.log['best_epoch'].append(np.asarray(self.solver.val_loss_history_avg).argmax())
    self.log['best_val_acc'].append(np.asarray(self.solver.val_loss_history_avg).max())

    # Delete Pseudo_labeled_samples
    self.log['current_train_indices'] = np.setdiff1d(self.log['current_train_indices'], self.current_pseudo_labeled_indices)
    self.log['current_train_set_percentage'].append(self.log['current_train_indices'].shape[0]/self.full_train_indices.shape[0])
    self._test_model(self.model, self.val_loader)

In [239]:
np.random.seed(RS)
torch.manual_seed(RS)
al_softmax._do_training(1, optim_args = {'lr': 0.0001})

starting training process.

training epoch [1]
Percent: [###################################] 100% train_loss: 5.026 train_acc: 8.33 Done...
loss_train_avg: 5.342, train_acc_avg: 6.236
starting validation process
Percent: [###################################] 100% Done...
loss_val_avg: 5.003, val_acc_avg: 8.338
Start testing model
Percent: [###################################] 100% Done...
Model scored 8.33% on Validation Set


In [240]:
np.random.seed(RS)
torch.manual_seed(RS)
al_random._do_training(1, optim_args = {'lr': 0.0001})

starting training process.

training epoch [1]
Percent: [###################################] 100% train_loss: 5.092 train_acc: 7.29 Done...
loss_train_avg: 5.344, train_acc_avg: 7.024
starting validation process
Percent: [###################################] 100% Done...
loss_val_avg: 4.983, val_acc_avg: 8.369
Start testing model
Percent: [###################################] 100% Done...
Model scored 8.37% on Validation Set
